# cesm Rainfall Anomalies

In [1]:
#import my functions
import helpers.fileHandler as fh
import utils._modelDefinitions as _model
import utils.timePeriod as tp

In [2]:
import xarray
import numpy
import cftime
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [3]:
secondsPerYear = 60*60*24*265


# Calculate a climatology

Based on the control run, calculate monthly anomalies

In [4]:
preclDa = fh.loadModelData('CESM-LME', 'PRECL', _model.cesmCntl)
preccDa = fh.loadModelData('CESM-LME', 'PRECC', _model.cesmCntl)


/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECL/
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECL/b.e11.B1850C5CN.f19_g16.0850cntl.001.cam.h0.PRECL.085001-184912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECL/b.e11.B1850C5CN.f19_g16.0850cntl.001.cam.h0.PRECL.185001-200512.nc']
/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECC/
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECC/b.e11.B1850C5CN.f19_g16.0850cntl.001.cam.h0.PRECC.085001-184912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECC/b.e11.B1850C5CN.f19_g16.0850cntl.001.cam.h0.PRECC.185001-200512.nc']


In [5]:
prectDa = secondsPerYear*1000*(preclDa.PRECL+preccDa.PRECC)
prectDa.name='prect'

In [6]:
monMeansDa=prectDa.groupby('time.month').mean(dim='time')

In [7]:
monMeansDa.where(
        (monMeansDa.lat>-50) & (monMeansDa.lat<0) & (monMeansDa.lon>100) & (monMeansDa.lon<170),
        drop=True
    ).load()

<xarray.DataArray 'prect' (month: 12, lat: 26, lon: 27)>
array([[[ 763.69805156,  757.75534318,  751.47624798, ...,
          885.25836442,  958.89796877,  889.5726255 ],
        [ 688.31789072,  675.6742535 ,  662.53451572, ...,
          866.44817055, 1027.83254365, 1044.94286848],
        [ 561.20867606,  540.20319927,  516.95852929, ...,
          805.82914213,  981.15950034, 1288.15417009],
        ...,
        [2861.6862604 , 2435.61184833, 2773.40120587, ...,
         1309.69478306, 1233.65941629, 1147.78878939],
        [2031.83800408, 1927.15172378, 2975.43715116, ...,
          923.26527926,  855.87297929,  785.68181271],
        [1823.60347603, 2817.30335115, 3094.85197986, ...,
          841.63462923,  781.48285288,  719.6440953 ]],

       [[ 781.18947878,  777.86764137,  775.88703932, ...,
          912.47195855,  989.3705431 ,  912.68660719],
        [ 670.12317429,  662.6142966 ,  657.23456494, ...,
          878.56376618, 1041.07527277, 1037.24311717],
        [ 534.73394347,  520.82015688,  504.4625048 , ...,
          808.61879906,  973.54823952, 1253.93449071],
...
        [2443.09180607, 2273.7430156 , 2248.62140411, ...,
         1204.10264652, 1143.59849667, 1083.13364176],
        [2188.30650996, 1876.10816905, 2466.68649598, ...,
          798.93587833,  736.17153084,  678.48953568],
        [1801.70642082, 2384.45331944, 2432.77487951, ...,
          624.89586268,  572.32863621,  520.27233451]],

       [[ 705.43318423,  695.22166309,  686.55058045, ...,
          822.62372035,  898.53537974,  837.71939166],
        [ 652.68828103,  637.23513959,  619.57863555, ...,
          825.27465101,  987.79656827,  984.51896041],
        [ 553.76785018,  529.23397362,  503.46063954, ...,
          793.67832023,  971.76363378, 1260.24188962],
        ...,
        [2632.50842435, 2336.91084747, 2641.30651775, ...,
         1114.66428148, 1072.03335499, 1023.57898123],
        [2090.66344273, 1820.1882523 , 2779.44246193, ...,
          751.91288186,  705.11663274,  657.49068273],
        [1786.56492944, 2609.5332498 , 2812.36629296, ...,
          640.88952357,  594.229914  ,  545.29371632]]])
Coordinates:
  * lat      (lat) float64 -48.32 -46.42 -44.53 -42.63 ... -4.737 -2.842 -0.9474
  * lon      (lon) float64 102.5 105.0 107.5 110.0 ... 160.0 162.5 165.0 167.5
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12

# Carve out an area of interest, and calculate anomalies

In [9]:

for experiment in _model.cesmFullForcings:
    
    #load it
    preclDa = fh.loadModelData('CESM-LME', 'PRECL', experiment)
    preccDa = fh.loadModelData('CESM-LME', 'PRECC', experiment)
    prectDa = secondsPerYear*1000*(preclDa.PRECL+preccDa.PRECC)
    prectDa.name='prect'
    
    #anom
    anomDa=prectDa.groupby('time.month')-monMeansDa
    anomDa.name='precAnom'
        
    #grab area around Australia
    domainDa=anomDa.where(
        (anomDa.lat>-50) & (anomDa.lat<0) & (anomDa.lon>100) & (anomDa.lon<170),
        drop=True
    )
    
    # convert to mm/year and store
    domainDa.to_netcdf('results/cesmMonthlyPrec/'+experiment+'.nc')
    
     #rcp85 extensions
    if experiment in ['002','003', '008', '009']: 
        
        #extensions
        preclDa = fh.loadModelData('CESM-LME', 'PRECL', 'LME.'+experiment)
        preccDa = fh.loadModelData('CESM-LME', 'PRECC', 'LME.'+experiment)
        extPrectDa = secondsPerYear*1000*(preclDa.PRECL+preccDa.PRECC)
        extPrectDa.name='prect'
        
        prectDa = xarray.concat([prectDa, extPrectDa], 'time')


        #anom
        anomDa=prectDa.groupby('time.month')-monMeansDa
        anomDa.name='precAnom'

        #grab area around Australia
        domainDa=anomDa.where(
            (anomDa.lat>-50) & (anomDa.lat<0) & (anomDa.lon>100) & (anomDa.lon<170),
            drop=True
        )

        # convert to mm/year and store
        domainDa.to_netcdf('results/cesmMonthlyPrec/LME.'+experiment+'.nc')
        
    
    

/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECL/
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECL/b.e11.BLMTRC5CN.f19_g16.001.cam.h0.PRECL.085001-184912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECL/b.e11.BLMTRC5CN.f19_g16.001.cam.h0.PRECL.185001-200512.nc']
/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECC/
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECC/b.e11.BLMTRC5CN.f19_g16.001.cam.h0.PRECC.085001-184912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECC/b.e11.BLMTRC5CN.f19_g16.001.cam.h0.PRECC.185001-200512.nc']


/Users/ajs/opt/anaconda3/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 578 times more chunks
  return self.array[key]


/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECL/
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECL/b.e11.BLMTRC5CN.f19_g16.002.cam.h0.PRECL.085001-184912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECL/b.e11.BLMTRC5CN.f19_g16.002.cam.h0.PRECL.185001-200512.nc']
/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECC/
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECC/b.e11.BLMTRC5CN.f19_g16.002.cam.h0.PRECC.085001-184912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECC/b.e11.BLMTRC5CN.f19_g16.002.cam.h0.PRECC.185001-200512.nc']


/Users/ajs/opt/anaconda3/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 578 times more chunks
  return self.array[key]


/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECL/
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECL/b.e11.BRCP85C5CN.f19_g16.LME.002.cam.h0.PRECL.200601-210012.nc']
/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECC/
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECC/b.e11.BRCP85C5CN.f19_g16.LME.002.cam.h0.PRECC.200601-210012.nc']


...

/Users/ajs/opt/anaconda3/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 578 times more chunks
  return self.array[key]


/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECL/
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECL/b.e11.BLMTRC5CN.f19_g16.012.cam.h0.PRECL.085001-184912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECL/b.e11.BLMTRC5CN.f19_g16.012.cam.h0.PRECL.185001-200512.nc']
/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECC/
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECC/b.e11.BLMTRC5CN.f19_g16.012.cam.h0.PRECC.085001-184912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECC/b.e11.BLMTRC5CN.f19_g16.012.cam.h0.PRECC.185001-200512.nc']


/Users/ajs/opt/anaconda3/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 578 times more chunks
  return self.array[key]


/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECL/
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECL/b.e11.BLMTRC5CN.f19_g16.013.cam.h0.PRECL.085001-184912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECL/b.e11.BLMTRC5CN.f19_g16.013.cam.h0.PRECL.185001-200512.nc']
/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECC/
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECC/b.e11.BLMTRC5CN.f19_g16.013.cam.h0.PRECC.085001-184912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PRECC/b.e11.BLMTRC5CN.f19_g16.013.cam.h0.PRECC.185001-200512.nc']


/Users/ajs/opt/anaconda3/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 578 times more chunks
  return self.array[key]


# Time averaging for the anomalies

In [11]:
monthlyAnomDa=xarray.concat([
    xarray.open_dataset('results/cesmMonthlyPrec/'+iExp+'.nc') for iExp in _model.cesmAll],
    'experiment')
    

monthlyAnomDa=monthlyAnomDa.assign({'experiment':_model.cesmAll})

warmSeasonAnomDa=tp.averageForTimePeriod(monthlyAnomDa)

warmSeasonAnomDa.to_netcdf('results/cesmPrecAnoms.nc')

# Do that all again for Ts

In [12]:
tsDa = fh.loadModelData('CESM-LME', 'TS', _model.cesmCntl)

monMeansDa=tsDa.TS.groupby('time.month').mean(dim='time')

monMeansDa.where(
        (monMeansDa.lat>-50) & (monMeansDa.lat<0) & (monMeansDa.lon>100) & (monMeansDa.lon<170),
        drop=True
    ).load()

/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/TS/
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/TS/b.e11.B1850C5CN.f19_g16.0850cntl.001.cam.h0.TS.085001-184912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/TS/b.e11.B1850C5CN.f19_g16.0850cntl.001.cam.h0.TS.185001-200512.nc']


<xarray.DataArray 'TS' (month: 12, lat: 26, lon: 27)>
array([[[282.3029 , 282.37982, 282.453  , ..., 285.12256, 285.29666,
         284.7596 ],
        [283.459  , 283.4972 , 283.5798 , ..., 286.4485 , 286.78424,
         286.2769 ],
        [284.62888, 284.65228, 284.7156 , ..., 287.77362, 288.0664 ,
         287.87802],
        ...,
        [300.6783 , 297.80142, 301.59268, ..., 302.07162, 302.01572,
         301.93628],
        [297.66537, 296.9991 , 301.1753 , ..., 301.60736, 301.49063,
         301.36142],
        [296.50525, 301.20428, 301.6382 , ..., 301.0947 , 300.9362 ,
         300.7725 ]],

       [[282.7254 , 282.80463, 282.88074, ..., 285.83698, 286.00516,
         285.44794],
        [283.92322, 283.95853, 284.0398 , ..., 287.20044, 287.55966,
         286.83868],
        [285.1558 , 285.1732 , 285.2386 , ..., 288.56027, 288.86633,
         288.47354],
...
        [300.4305 , 298.107  , 302.19376, ..., 301.91733, 301.91766,
         301.88547],
        [297.88455, 297.2016 , 301.8522 , ..., 301.48654, 301.40683,
         301.33057],
        [296.87653, 302.21036, 302.53326, ..., 301.00912, 300.89197,
         300.77475]],

       [[281.152  , 281.22144, 281.2946 , ..., 283.62625, 283.7778 ,
         283.2848 ],
        [282.34995, 282.37405, 282.44168, ..., 284.8917 , 285.18594,
         284.7441 ],
        [283.42715, 283.44675, 283.4926 , ..., 286.09814, 286.3729 ,
         286.20593],
        ...,
        [300.74658, 297.96945, 302.2252 , ..., 302.01318, 301.9762 ,
         301.9234 ],
        [297.82126, 297.0872 , 301.7551 , ..., 301.58798, 301.49747,
         301.39532],
        [296.7144 , 301.92868, 302.28696, ..., 301.1211 , 300.98816,
         300.84598]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 -48.32 -46.42 -44.53 -42.63 ... -4.737 -2.842 -0.9474
  * lon      (lon) float64 102.5 105.0 107.5 110.0 ... 160.0 162.5 165.0 167.5
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12

# Carve out an area of interest, and calculate anomalies

In [14]:

for experiment in _model.cesmFullForcings:
    
    #load it
    tsDa = fh.loadModelData('CESM-LME', 'TS', experiment).TS
    
    #anom
    anomDa=tsDa.groupby('time.month')-monMeansDa
    anomDa.name='tsAnom'
        
    #grab area around Australia
    domainDa=anomDa.where(
        (anomDa.lat>-50) & (anomDa.lat<0) & (anomDa.lon>100) & (anomDa.lon<170),
        drop=True
    )
    
    # convert to mm/year and store
    domainDa.to_netcdf('results/cesmMonthlyTs/'+experiment+'.nc')
    
     #rcp85 extensions
    if experiment in ['002','003', '008', '009']: 
        
        #extensions
        extTsDa = fh.loadModelData('CESM-LME', 'TS', 'LME.'+experiment)
        tsDa=xarray.concat([tsDa, extTsDa.TS], 'time')

        #anom
        anomDa=tsDa.groupby('time.month')-monMeansDa
        anomDa.name='tsAnom'

        #grab area around Australia
        domainDa=anomDa.where(
            (anomDa.lat>-50) & (anomDa.lat<0) & (anomDa.lon>100) & (anomDa.lon<170),
            drop=True
        )

        # convert to mm/year and store
        domainDa.to_netcdf('results/cesmMonthlyTs/LME.'+experiment+'.nc')
        
    
    

/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/TS/
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/TS/b.e11.BLMTRC5CN.f19_g16.001.cam.h0.TS.085001-184912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/TS/b.e11.BLMTRC5CN.f19_g16.001.cam.h0.TS.185001-200512.nc']


/Users/ajs/opt/anaconda3/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 578 times more chunks
  return self.array[key]


/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/TS/
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/TS/b.e11.BLMTRC5CN.f19_g16.002.cam.h0.TS.085001-184912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/TS/b.e11.BLMTRC5CN.f19_g16.002.cam.h0.TS.185001-200512.nc']


/Users/ajs/opt/anaconda3/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 578 times more chunks
  return self.array[key]


/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/TS/
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/TS/b.e11.BRCP85C5CN.f19_g16.LME.002.cam.h0.TS.200601-210012.nc']


...

/Users/ajs/opt/anaconda3/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 578 times more chunks
  return self.array[key]


/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/TS/
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/TS/b.e11.BLMTRC5CN.f19_g16.012.cam.h0.TS.085001-184912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/TS/b.e11.BLMTRC5CN.f19_g16.012.cam.h0.TS.185001-200512.nc']


/Users/ajs/opt/anaconda3/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 578 times more chunks
  return self.array[key]


/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/TS/
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/TS/b.e11.BLMTRC5CN.f19_g16.013.cam.h0.TS.085001-184912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/TS/b.e11.BLMTRC5CN.f19_g16.013.cam.h0.TS.185001-200512.nc']


/Users/ajs/opt/anaconda3/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 578 times more chunks
  return self.array[key]


# Time averaging for the anomalies

In [15]:
monthlyAnomDa=xarray.concat([
    xarray.open_dataset('results/cesmMonthlyTs/'+iExp+'.nc') for iExp in _model.cesmAll],
    'experiment')
    

monthlyAnomDa=monthlyAnomDa.assign({'experiment':_model.cesmAll})

warmSeasonAnomDa=tp.averageForTimePeriod(monthlyAnomDa)

warmSeasonAnomDa.to_netcdf('results/cesmTsAnoms.nc')